# Main Thesis Topic: “Zero-shot classification of ECG signals using CLIP-like model”.

**For example: Train on PBT-XL:**

- Text Encoder: ClinicalBERT (trained on diagnoses of ECG signal to obtain corresponding embeddings)
- Image Encoder: 1D-CNN (used to encode ECG signal to obtain signal embeddings)

- Experiment A): Baseline: We can take only the name of the class. For example, take “Myocardial Infarction” as a text. We should exclude some classes from training and after training is completed, the CLIP-like model can be tested on these excluded classes.
    - Next, we get embeddings of text from ClinicalBERT and train the ECG encoder with contrastive loss.

- Experiment B): Same as Experiment A but instead of testing on the same dataset/classes, we would test on other datasets containing different classes.

**Evaluation metrics:**
- Main: AUC-ROC, average_precison_score,
- Optional: Specificity, Sensitivity, F1-score

**Outcome:**
- It’s possible to train CLIP-like models with freezed (or unchanged/not fine tuned for downstream tasks) text encoder
- Training ECG encoders that are viable for representing different domains (within ECG modality) and previously unseen classes.
- Training a CLIP-like model on ECGs has little novelty.

First, we preprocess the ECG data from the PhysioNet 2021 challenge dataset. This data will be loaded using the ```PhysioNetDataset``` class.

In [1]:
pip install transformers

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
from scipy.signal import resample
import torch
from transformers import AutoTokenizer, AutoModel
import ast
import scipy.io as sio
from torch.utils.data import random_split

In [3]:
import torch
import torch.nn as nn

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
PyFiles_PATH = '/content/drive/MyDrive/ECG Project (Shared Folder)/PyFiles'
PyFiles_PATH

'/content/drive/MyDrive/ECG Project (Shared Folder)/PyFiles'

In [6]:
sys.path.append(PyFiles_PATH)

In [7]:
from helper_functions import *
from dataset import PhysioNetDataset

In [8]:
# Path to training folder within PhysioNet dataset
PhysioNet_PATH = '/content/drive/MyDrive/ECG Project (Shared Folder)/Datasets/physionet.org/files/challenge-2021/1.0.3/training'
PhysioNet_PATH

'/content/drive/MyDrive/ECG Project (Shared Folder)/Datasets/physionet.org/files/challenge-2021/1.0.3/training'

Using the ```PhysioNet_PATH```, we can create separate datasets for training, testing & validation.

# Stage 1: Data Preprocessing

- train_set (train & validation data)
- test_set (test data)

Google CPU is quite slow when processing the for-loops below so if you have a decent CPU, you can try connecting to local run time via Jupyter Lab for this portion of the pipeline only.

```
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0
```

In [9]:
train_set = PhysioNetDataset(PhysioNet_PATH, train=True)
test_set = PhysioNetDataset(PhysioNet_PATH, train=False)

len(train_set), len(test_set)

(66167, 22352)

The ```train_set``` can be split into ```current_train``` (85%) and ```current_val``` (15%).

In [10]:
# Set the seed for the random number generator
torch.manual_seed(0)

# Get the length of the train_set
length = len(train_set)

# Calculate the lengths of the splits
train_length = int(0.85 * length)
val_length = length - train_length

# Split the dataset
current_train, current_val = random_split(train_set, [train_length, val_length])

The next step is to extract the header data for ```current_train```, ```current_val```, and ```test_set``` and save the data to a csv file.

## current_train

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(current_train)), desc="Processing records"):
    record, _ = current_train[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('train_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

## current_val

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(current_val)), desc="Processing records"):
    record, _ = current_val[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('val_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

## test_set

In [ ]:
# Initialize an empty list to store the records
records = []

# Iterate over all records
for i in tqdm(range(len(test_set)), desc="Processing records"):
    record, _ = test_set[i]  # Get the record (ignore the ECG data for now)

    # Flatten the 'leads_info' list into separate columns for each lead
    for j, lead_info in enumerate(record['leads_info']):
        for key, value in lead_info.items():
            record[f'lead_{j}_{key}'] = value
    del record['leads_info']  # We don't need the 'leads_info' list anymore

    # Append the record to the list
    records.append(record)

# Convert the list of records into a DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv('test_set_records.csv', index=False)

print(f"Processed {len(records)} records.")

Now that the header data has been extracted and saved to csv files, we can map the corresponding SNOWMED-CT code to the csv files too.

First, let's load the SNOWMED-CT mappings:

In [11]:
smowmed_mappings_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/SNOWMED-CT Codes/combined_mappings.csv'

# Load the SNOMED-CT mappings
smowmed_mappings = pd.read_csv(smowmed_mappings_path)
smowmed_mappings.head(2)

,Dx,SNOMEDCTCode,Abbreviation,CPSC,CPSC_Extra,StPetersburg,PTB,PTB_XL,Georgia,Chapman_Shaoxing,Ningbo,Total,Notes
0,atrial fibrillation,164889003,AF,1221,153,2,15,1514,570,1780,0,5255,NaN
1,atrial flutter,164890007,AFL,0,54,0,1,73,186,445,7615,8374,NaN


In [12]:
# Select the 'Dx' and 'SNOMEDCTCode' columns
codes = smowmed_mappings[['Dx', 'SNOMEDCTCode']]

# Set 'SNOWMEDCTCode' as the index
codes.set_index('SNOMEDCTCode', inplace=True)

# Convert the DataFrame into a dictionary
codes_dict = codes['Dx'].to_dict()

In [13]:
len(codes_dict)

133

Now, let's load the csv files and map the corresponding codes from ```codes_dict``` to the csv files:

In [14]:
train_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/train_set_records.csv'
val_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/val_set_records.csv'
test_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/test_set_records.csv'

In [15]:
train_set_df = load_and_process(train_set_path)
val_set_df = load_and_process(val_set_path)
test_set_df = load_and_process(test_set_path)

Now, using the ```map_codes_to_dx()``` function, let's map the SNOWMED-CT codes for each ECG signal ```dx```. The new column containing the diagnosis name will be ```dx_modality```

In [16]:
def map_codes_to_dx(codes):
    return [codes_dict.get(int(code), code) for code in codes]

In [17]:
train_set_df['dx_modality'] = train_set_df['dx'].apply(map_codes_to_dx)

In [18]:
val_set_df['dx_modality'] = val_set_df['dx'].apply(map_codes_to_dx)

In [19]:
test_set_df['dx_modality'] = test_set_df['dx'].apply(map_codes_to_dx)

In [21]:
# Example:
test_set_df['dx_modality'][500]

['sinus rhythm']

Now, let's save these to new csv files so that they contain the new `dx_modality` column:

- `processed_train_set_records.csv`
- `processed_val_set_records.csv`
- `processed_test_set_records.csv`

In [22]:
train_set_df.to_csv('processed_train_set_records.csv', index=False)

In [23]:
val_set_df.to_csv('processed_val_set_records.csv', index=False)

In [24]:
test_set_df.to_csv('processed_test_set_records.csv', index=False)

# Stage 2: ECG Classification Model Pipeline

Now that our data is preprocessed, we can begin working on the Model Pipeline itself. The ECG Classification Model Pipeline will consist of three components:

1. `TextEncoder()` class

2. `ECGEncoder()` class

3. `CLIPModel()` class

An overview and outline of each of these components can be found below in their respective subsections.

First, let's load the csv files that contain information about our ECG header data.

`NOTE: The number of records in each csv files should match the number of records in current_train, current_val, and test_set, respectively`

In [14]:
processed_train_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_train_set_records.csv'
processed_val_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_val_set_records.csv'
processed_test_set_path = '/content/drive/MyDrive/ECG Project (Shared Folder)/Data/PhysioNet/processed_test_set_records.csv'

In [15]:
processed_train_df = pd.read_csv(processed_train_set_path)
processed_val_df = pd.read_csv(processed_val_set_path)
processed_test_df = pd.read_csv(processed_test_set_path)

In [28]:
print("There are {} records in the current_train and {} records in the processed_train_df.".format(len(current_train), len(processed_train_df)))

There are 56241 records in the current_train and 56015 records in the processed_train_df.


In [25]:
print("There are {} records in the val_set and {} records in the processed_val_df.".format(len(current_val), len(processed_val_df)))

There are 9926 records in the val_set and 9885 records in the processed_val_df.


In [27]:
print("There are {} records in the test_set and {} records in the processed_test_df.".format(len(test_set), len(processed_test_df)))

There are 22352 records in the test_set and 22352 records in the processed_test_df.


## TextEncoder()

Create a class, ```TextEncoder()``` that is used to convert the description of the (dx_modality) diagnosis class into an embeddings using the pretrained, base ClinicalBERT model.

- Input should be a concatenated using comma or blank space string of diagnoses/dx_modality per ECG signal.
- Use processed CSV files (dx_modality only)
- Frozen weights (since it's already pretrained)

In [19]:
class TextEncoder:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        self.model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

    def encode(self, text_list):
        # Check if text_list is a string representation of a list
        if isinstance(text_list, str):
            text_list = ast.literal_eval(text_list)
        # Convert list of strings to a single string
        text = ', '.join(text_list)
        # Tokenize text
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        # Get embeddings from ClinicalBERT model
        with torch.no_grad():
            embeddings = self.model(**inputs).last_hidden_state
        # Average the embeddings to get single vector per each input
        embeddings = torch.mean(embeddings, dim=1)
        return embeddings

In [20]:
text_encoder = TextEncoder()
embeddings = text_encoder.encode(processed_train_df['dx_modality'][0])
print(embeddings.size())

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

torch.Size([1, 768])


In [ ]:
embeddings

## ECGEncoder()

- Input is ECG signal, output will be embeddings of ECG signal
- This is going to be model in model.py
- Model weights are updated iteratively
- optimizer = torch.optim.Adam(clip_model.ECGEncoder.parameters())

In [56]:
#import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [51]:
class OneDimCNN(nn.Module):
    def __init__(self, num_classes):
        super(OneDimCNN, self).__init__()

        # Layer 1
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 2
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 3
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm1d(64)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 4
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 5
        self.conv5 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm1d(256)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 6
        self.conv6 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm1d(512)
        self.relu6 = nn.ReLU()
        self.pool6 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 7
        self.conv7 = nn.Conv1d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm1d(1024)
        self.relu7 = nn.ReLU()
        self.pool7 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 8
        self.conv8 = nn.Conv1d(in_channels=1024, out_channels=2048, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm1d(2048)
        self.relu8 = nn.ReLU()
        self.pool8 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Layer 9
        self.conv9 = nn.Conv1d(in_channels=2048, out_channels=4096, kernel_size=3, stride=1, padding=1)
        self.bn9 = nn.BatchNorm1d(4096)
        self.relu9 = nn.ReLU()
        self.pool9 = nn.AvgPool1d(kernel_size=2, stride=2)

        # Fully Connected Layer 1
        self.fc1 = nn.Linear(4096, 128)  # Adjusted to match output channels of last conv layer
        self.relu10 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        # Fully Connected Layer 2
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Layer 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # Layer 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # Layer 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Layer 4
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool4(x)

        # Layer 5
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.pool5(x)

        # Layer 6
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.pool6(x)

        # Layer 7
        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu7(x)
        x = self.pool7(x)

        # Layer 8
        x = self.conv8(x)
        x = self.bn8(x)
        x = self.relu8(x)
        x = self.pool8(x)

        # Layer 9
        x = self.conv9(x)
        x = self.bn9(x)
        x = self.relu9(x)
        x = x.view(x.size(0), -1)

        # Dynamically adjust the input size of the first fully connected layer
        if self.fc1.in_features != x.size(1):
            self.fc1 = nn.Linear(x.size(1), 128)

        # Fully Connected Layer 1
        x = self.fc1(x)
        x = self.relu10(x)  # Renamed from relu5 to relu10
        x = self.dropout1(x)

        # Fully Connected Layer 2
        x = self.fc2(x)

        return x

In [52]:
class ECGEncoder(OneDimCNN):
    def __init__(self, num_classes):
        super(ECGEncoder, self).__init__(num_classes)

    def encode(self, signal):
        # Encode the signal using the OneDimCNN model.
        encoded_signal = self.forward(signal)

        # Return the encoded signal.
        return encoded_signal

In [43]:
type(train_set[0][1]['val'])

numpy.ndarray

In [44]:
train_set[0][1]['val']

array([[ -49.41174209,  -49.41174209,  -49.41174209, ...,    3.26986681,
           6.65822132,    4.35320015],
       [  39.23760431,   39.23760431,   39.23760431, ...,  -45.89316397,
         -44.94818514,  -42.39476713],
       [  95.17413779,   95.17413779,   95.17413779, ..., -112.12670284,
        -112.96003617, -116.70843301]])

In [73]:
# Define the number of classes
num_classes = len(codes_dict)

# Create an instance of the model
ecg_encoder = ECGEncoder(num_classes)

# Convert the numpy array to a PyTorch tensor
input_data = torch.from_numpy(train_set[40000][1]['val']).float()

# Add an extra dimension for the batch size
input_data = input_data.unsqueeze(0)
# Convert the model's weights to Float
ecg_encoder = ecg_encoder.float()

# Pass the data through the model
output = ecg_encoder(input_data)

print(output)

tensor([[ 0.0759,  0.3611,  0.1698,  0.0342, -0.0232,  0.0714,  0.2897,  0.0645,
          0.3557,  0.1778,  0.0551, -0.2179,  0.2190, -0.0648, -0.0200, -0.2931,
         -0.3637, -0.0882, -0.0564, -0.4627,  0.0592,  0.0682,  0.1784,  0.0803,
         -0.3192, -0.1793,  0.0423, -0.1921,  0.0630,  0.0969,  0.2411,  0.2804,
          0.1373, -0.0132,  0.2371, -0.0876, -0.2263, -0.2970,  0.2803, -0.2026,
          0.1714,  0.2798,  0.0293,  0.2405,  0.2005,  0.1016,  0.0962, -0.0771,
         -0.0614,  0.4581,  0.1764, -0.0372,  0.1472,  0.1202,  0.3661,  0.4227,
         -0.2465,  0.3431,  0.1962,  0.2239, -0.0369, -0.0260, -0.2162, -0.0492,
          0.1254, -0.0775, -0.0228, -0.0084, -0.1755, -0.0842,  0.0135,  0.0937,
         -0.4402,  0.0572,  0.4507, -0.2793,  0.1144, -0.1732,  0.0356,  0.2844,
         -0.2312,  0.2778,  0.0634, -0.2775,  0.0965,  0.2280, -0.0395,  0.2462,
         -0.0641, -0.1115, -0.1430, -0.3193,  0.4543,  0.0065, -0.5322,  0.1574,
          0.4193,  0.0185, -

In [69]:
batch_size = input_data.shape[0]
num_channels = input_data.shape[1]
tensor_size = input_data[0][2]

print("Batch size:", batch_size)
print("Number of channels:", num_channels)
print(f"Tensor size: {len(tensor_size)}")

Batch size: 1
Number of channels: 3
Tensor size: 5000


In [75]:
# Convert the model's weights to Float
ecg_encoder = ecg_encoder.float()

# Set the model in evaluation mode
ecg_encoder.eval()

# Pass the data through the model
output = ecg_encoder(input_data)

print(output)

tensor([[ 0.0366,  0.0413, -0.0027, -0.0578, -0.0053, -0.0372, -0.0181,  0.0460,
         -0.0192,  0.0500, -0.0354,  0.0205,  0.0545,  0.0223,  0.0636,  0.0792,
         -0.0749, -0.0766, -0.0679,  0.0678,  0.0859,  0.0025,  0.0142, -0.0305,
         -0.0153, -0.0110,  0.0350, -0.0598, -0.0245, -0.0544,  0.0573, -0.0540,
         -0.0338, -0.0214, -0.0253, -0.0305, -0.0524, -0.0605,  0.0604, -0.0099,
          0.0389,  0.0002, -0.0543,  0.0288,  0.0761, -0.0324,  0.0718, -0.0424,
          0.0412,  0.0709,  0.0601, -0.0162, -0.0564,  0.0087, -0.0253, -0.0584,
         -0.0498,  0.0884, -0.0535, -0.0689, -0.0565, -0.0149, -0.0467, -0.0924,
          0.0671, -0.0572,  0.0053,  0.0336,  0.0249,  0.0417,  0.0266,  0.0101,
         -0.0761, -0.0457, -0.0347, -0.0718,  0.0036,  0.0237,  0.0556,  0.0801,
          0.0395,  0.0838,  0.0624,  0.0123, -0.0511, -0.0340,  0.0590,  0.0749,
          0.0732, -0.0208, -0.0642,  0.0670,  0.0203, -0.0598,  0.0510, -0.0019,
          0.0289,  0.0011,  

## InstanceSelecter()

- negative_instances are where these two embeddings do not align
- filter out text embeddings that are the same or equal to the positive_instances

Update InstanceSelector such that there are is no positive instance function and generate false_instances based on instances that do not correspond


In [ ]:
class InstanceSelector:
    def __init__(self, train_set, processed_train_df, text_encoder, ecg_encoder):
        self.train_set = train_set
        self.processed_train_df = processed_train_df
        self.text_encoder = text_encoder
        self.ecg_encoder = ecg_encoder

    def get_positive_instances(self):
        positive_instances = []
        for i in tqdm(range(len(self.train_set)), desc="Generating positive instances"):
            ecg_embedding = self.ecg_encoder.encode(self.train_set[i][1]['val'])
            dx_modality_embedding = self.text_encoder.encode(self.processed_train_df['dx_modality'][i])
            if torch.all(torch.eq(ecg_embedding, dx_modality_embedding)):
                positive_instances.append((ecg_embedding, dx_modality_embedding))
        return positive_instances

    def get_negative_instances(self):
        negative_instances = []
        positive_instances = self.get_positive_instances()
        for i in tqdm(range(len(self.train_set)), desc="Generating negative instances"):
            ecg_embedding = self.ecg_encoder.encode(self.train_set[i][1]['val'])
            for j in range(len(self.processed_train_df)):
                if i != j:
                    dx_modality_embedding = self.text_encoder.encode(self.processed_train_df['dx_modality'][j])
                    if not any(torch.all(torch.eq(ecg_embedding, pos[1])) for pos in positive_instances):
                        negative_instances.append((ecg_embedding, dx_modality_embedding))
        return negative_instances

In [ ]:
text_encoder = TextEncoder()
ecg_encoder = ECGEncoder(num_classes=126)  # Assuming you have this class defined

In [ ]:
instance_selector = InstanceSelector(train_set, processed_train_df, text_encoder, ecg_encoder)

In [ ]:
positive_instances = instance_selector.get_positive_instances()

In [ ]:
negative_instances = instance_selector.get_negative_instances()

# CLIPModel


In [ ]:
class CLIPModel(nn.Module):
    def __init__(self, train_set, processed_train_df):
        super(CLIPModel, self).__init__()
        self.ecg_encoder = ECGEncoder(num_classes=126)  # Initialize ECGEncoder
        self.text_encoder = TextEncoder()  # Initialize TextEncoder
        self.instance_selector = InstanceSelector(train_set, processed_train_df, self.text_encoder, self.ecg_encoder)

    def forward(self, ecgs, diagnoses):
        ecgs_embeddings = self.ecg_encoder(ecgs)
        diagnoses_embeddings = self.text_encoder.encode(diagnoses)
        positive_instances = self.instance_selector.get_positive_instances()
        negative_instances = self.instance_selector.get_negative_instances()
        # Compute loss based on whether the pair of embeddings is a positive or negative instance
        loss = sum(F.cosine_similarity(ecgs_embeddings[i], diagnoses_embeddings[i]) for i in range(len(ecgs)) if (ecgs_embeddings[i], diagnoses_embeddings[i]) in positive_instances) \
             - sum(F.cosine_similarity(ecgs_embeddings[i], diagnoses_embeddings[i]) for i in range(len(ecgs)) if (ecgs_embeddings[i], diagnoses_embeddings[i]) in negative_instances)
        return loss

Example of how the CLIP-like model works:

Creating small data subsets for training example:

- 100 records only for train_set
- 100 records only for processed_train_df

In [ ]:
sample_train_set_100 = train_set[:100]
sample_processed_train_df_100 = processed_train_df.iloc[:100]

In [ ]:
len(sample_processed_train_df_100) , len(sample_train_set_100)

(100, 100)

In [ ]:
# Initialize model
model = CLIPModel(sample_train_set_100, sample_processed_train_df_100)
# Initialize optimizer
optimizer = torch.optim.Adam(model.ecg_encoder.parameters())

num_params = sum(p.numel() for p in model.parameters())
print("Number of parameters: ", num_params)

Number of parameters:  10468286


In [ ]:
# Training params
num_epochs = 3

In [ ]:
# Initialize a list to store the loss at each step
losses = []

# Training loop
for epoch in range(num_epochs):
    # Add a progress bar for the inner loop
    for i in tqdm(range(len(sample_train_set_100)), desc=f"Training epoch {epoch+1}/{2}"):
        # Get ECGs and diagnoses from training set
        ecgs = sample_train_set_100[i][1]['val']
        diagnoses = sample_processed_train_df_100['dx_modality'][i]

        # Convert ECGs to tensor and add a dimension for batch size
        ecgs = torch.from_numpy(ecgs).float().unsqueeze(0)

        # Forward pass
        loss = model(ecgs, diagnoses)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Save the loss to a variable
        losses.append(loss.item())

    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Save the model checkpoint after training
torch.save(model.state_dict(), 'model.ckpt')

Generating positive instances: 100%|██████████| 100/100 [00:14<00:00,  6.89it/s]

Generating positive instances: 100%|██████████| 100/100 [00:17<00:00,  5.70it/s]

Training epoch 1/2:   0%|          | 0/100 [18:34<?, ?it/s]


RuntimeError: ignored

```
class CLIPModel(nn.Module):
def
def __init__(self, ):
Konstantin Egorov8:28 AM
class CLIPModule(nn.Module):
	def __init__(self, ):
		self.ecg_encoder = ECGEncoder()
		self.text_encoder = TextEncoder()
		self.triplet_loss = TripletLoss()

	def forward(self, ecgs, diagnoses):
		ecgs_embeddings = self.ecg_encoder(ecgs)
		diagnoses_embeddings = self.text_encoder(diagnoses)
		loss – self.triplet_loss(ecgs_embeddings, diagnoses_embeddings)
```